# CIFAR-10 Image Classification 

## References
###https://www.mikulskibartosz.name/pca-how-to-choose-the-number-of-components/
###https://towardsdatascience.com/hog-histogram-of-oriented-gradients-67ecd887675f
###https://ieeexplore.ieee.org/document/8971585
###https://www.kaggle.com/code/manikg/training-svm-classifier-with-hog-features
###https://towardsdatascience.com/principal-component-analysis-pca-explained-visually-with-zero-math-1cbf392b9e7d
###Andrew NG - Dimensionality Reduction

In [ ]:
# Import Libraries

import numpy
import time
import calendar
from six.moves import cPickle as pickle
import numpy as np
import cv2
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix
from skimage.io import imread
from skimage.transform import resize
from skimage.feature import hog
from skimage import exposure
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.metrics import classification_report,accuracy_score

##### General parameters

In [ ]:
# General Parameters

imageSize = 32  
channels = 3    
classes = 10    
trainingDataSize = 50000    
testDataSize = 10000        
trainingDataFiles = ('./dataset/cifar-10-batches-py/data_batch_1', './dataset/cifar-10-batches-py/data_batch_2', './dataset/cifar-10-batches-py/data_batch_3', './dataset/cifar-10-batches-py/data_batch_4','./dataset/cifar-10-batches-py/data_batch_5') 
testDataFile = './dataset/cifar-10-batches-py/test_batch' 
pcaFileName = 'pca' 
svmFileName = 'svm' 

##### Load the data from the given filename

In [ ]:
def loadData(filename):
    '''
    Load the data from the given filename

    Parameters:
    -----------
    filename: string
        The name of the file containing the data to load

    Returns:
    --------
    theSet['data']:     array of images
    theSet['labels']:   array of labels
    '''
    f = open(filename, 'rb')
    theSet = pickle.load(f,encoding='latin1')
    f.close()

    return theSet['data'], theSet['labels']

##### Convert the images from CIFAR-10 format, to an array with each image of 32 X 32 X 3 size

In [ ]:
def convertImages(origImages):
    '''
    Convert the images from CIFAR-10 format, to an array of 10000 images each is 32 X 32 X 3 size

    Parameters:
    -----------
    origImages: array
        array of images in the CIFAR-10 format

    Returns:
    --------
    images:     array of images each in 32 X 32 X 3 size
    '''
    images = np.reshape(origImages,(-1, channels, imageSize, imageSize))
    images = np.transpose(images, (0,2,3,1))

    return images

##### Load all the training data from all files

In [ ]:
def loadTrainingData(filenames):
    '''
    Load all the training data from all files

    Parameters:
    -----------
    filenames: array of string
        An array The name of the file containing the data to load

    Returns:
    --------
    trainingImages: array of the training set images
    trainingLabels: array of the training set labels
    '''

    #Pre-allocate the arrays
    trainingImages = np.zeros(shape=[trainingDataSize, imageSize, imageSize, channels], dtype=numpy.uint8)
    trainingLabels = np.zeros(shape=[trainingDataSize], dtype=int)

    start=0
    for fileName in filenames:

      origImages, labels = loadData(fileName)
      images = convertImages(origImages)

      numOfImages = len(images)
      end = start + numOfImages
      # print(numOfImages)
      trainingImages[start:end, :] = images
      trainingLabels[start:end] = labels
      start = end

    return trainingImages, trainingLabels

##### Load the test data

In [ ]:
def loadTestData(filename):
    '''
    Load the test data

    Parameters:
    -----------
    filename: string
        The name of the file containing the test data

    Returns:
    --------
    testImages: array of images of the test data
    testLabels: array of labels of the test data
    '''

    origTestImages, testLabels = loadData(filename)
    testImages = convertImages(origTestImages)

    return testImages, testLabels

##### Returns the current time in seconds since epoch

In [ ]:
def currentTime():
    '''
    Returns the current time in seconds since EPOC
    Used to measure how much time each phase took

    Returns:
    --------
    the current time in second since EPOC
    '''

    return calendar.timegm(time.gmtime())

##### Calculate the HOG descriptors of the given images

In [ ]:
winSize = imageSize
blockSize = 12
blockStride = 4
cellSize = 4
nbins = 9
derivAperture = 1
winSigma = -1.
histogramNormType = 0
L2HysThreshold = 0.2
gammaCorrection = True
nlevels = 64
signedGradient = True
hog = cv2.HOGDescriptor((winSize,winSize),(blockSize, blockSize),(blockStride,blockStride),(cellSize,cellSize),nbins,derivAperture, winSigma,histogramNormType,L2HysThreshold,gammaCorrection,nlevels,signedGradient)

In [ ]:
# HOG for all images in image set

def calcHOG(images):
  '''
    Calculate the HOG descriptors of the given images

    Parameters:
    -----------
    images: an array of images
        The images to which a HOG calculation should be applied

    Returns:
    --------
    hogDescriptors: an array of HOG vectors, 5832 components each
    '''
    
  hogDescriptors = []
  for image in images:
    hogDescriptors.append( hog.compute(image) )

  hogDescriptors = np.squeeze(hogDescriptors)
  return hogDescriptors

#### Algorithm Implementation

##### Training Phase

###### First load the data into two arrays:

In [ ]:
from google.colab import drive
drive.mount('/content/Drive', force_remount=True)

Mounted at /content/Drive


In [ ]:
!mkdir -p "/content/dataset"
!tar -xzf "/content/Drive/MyDrive/CIFAR-10/cifar-10-python.tar.gz" -C "/content/dataset"

In [ ]:
print("Loading the training set..."),
tik = currentTime()
trainingImages, trainingLabels = loadTrainingData(trainingDataFiles)
print("Took: " + str(currentTime()-tik) + " sec" )

Loading the training set...
Took: 0 sec


###### Create a HOG descriptor from these images

In [ ]:
print("Creating HOG descriptors from the training set..."),
tik = currentTime()
trainHogDescriptors = calcHOG(trainingImages)
print("Took: " + str(currentTime() - tik) + " sec")

Creating HOG descriptors from the training set...
Took: 4 sec


###### Reduce the dimension of the HOG descriptors using PCA

In [ ]:
print("Reducing dimension of the HOG descriptors " + "..."),
tik = currentTime()
pca = PCA(n_components=0.99)
trainHogProjected = pca.fit_transform(trainHogDescriptors)
print("Took: " + str(currentTime() - tik) + " sec")

Reducing dimension of the HOG descriptors ...
Took: 68 sec


###### Save it as a pca file

In [ ]:
print("Save it as a PCA file..."),
tik = currentTime()
pcaFile = open(pcaFileName, 'wb')
pickle.dump(pca, pcaFile)
pcaFile.close()
print("Took: " + str(currentTime() - tik) + " sec")

Save it as a PCA file...
Took: 0 sec


###### Train the SVM model using the reduced HOG descriptor

In [ ]:
svmm = svm.SVC()
svmm.fit(trainHogProjected,trainingLabels)
print("Save it as a SVM file..."),
tik = currentTime()
svmFile = open(svmFileName, 'wb')
pickle.dump(svmm, svmFile)
svmFile.close()
print("Took: " + str(currentTime() - tik) + " sec")

Save it as a SVM file...
Took: 2 sec


##### Testing phase

###### Load the test set

In [ ]:
print("Loading the test set..."),
tik = currentTime()
testImages, testLabels = loadTestData(testDataFile)
print("Took: " + str(currentTime() - tik) + " sec")

Loading the test set...
Took: 0 sec


###### Create HOG descriptors from the test set

In [ ]:
print("Creating HOG descriptors from the test set..."),
tik = currentTime()
testHogDescriptors = calcHOG(testImages)
print("Took: " + str(currentTime() - tik) + " sec")

Creating HOG descriptors from the test set...
Took: 1 sec


###### Reduce the dimension of the HOG descriptors

In [ ]:
print("Reducing the dimension of the HOG descriptors " + "..."),
tik = currentTime()
testHogProjected = pca.transform(testHogDescriptors)
print("Took: " + str(currentTime() - tik) + " sec")

Reducing the dimension of the HOG descriptors ...
Took: 2 sec


###### Classify the test set

In [ ]:
print("Classifying the test set..."),
tik = currentTime()
# testResponse = svm.predict(np.asarray(testHogProjected))[1].ravel()
testResponse = svmm.predict(testHogProjected)
print("Took: " + str(currentTime() - tik) + " sec")

Classifying the test set...
Took: 676 sec


###### Calculate the confusion matrix

In [ ]:
print ("Confusion matrix:")
confusionMatrix = confusion_matrix(testLabels, testResponse)
print(confusionMatrix)

Confusion matrix:
[[764  11  60  21  23   8  15   6  61  31]
 [ 16 831  10  13  11   6  16   3  39  55]
 [ 74   6 556  76 114  67  50  27  17  13]
 [ 31  12  68 542  85 130  61  36  14  21]
 [ 30   5  67  73 666  38  48  52  16   5]
 [ 19   5  61 159  64 595  36  49   6   6]
 [ 20  17  32  74  42  25 770   2  10   8]
 [ 17   3  41  49  68  53   6 728  16  19]
 [ 56  35  20  13  15   7   8   7 812  27]
 [ 34  64  12  21  11  11  11  17  23 796]]


###### Calculate accuracy

In [ ]:
accuracy = accuracy_score(testLabels, testResponse)
print("Percentage Accuracy: %.2f %%" % (accuracy*100))

Percentage Accuracy: 70.60 %


###### Classification Report

In [ ]:
print('Classification Report:')
clf_report = classification_report(testLabels,testResponse)
print(clf_report)

Classification Report:
              precision    recall  f1-score   support

           0       0.72      0.76      0.74      1000
           1       0.84      0.83      0.84      1000
           2       0.60      0.56      0.58      1000
           3       0.52      0.54      0.53      1000
           4       0.61      0.67      0.63      1000
           5       0.63      0.59      0.61      1000
           6       0.75      0.77      0.76      1000
           7       0.79      0.73      0.76      1000
           8       0.80      0.81      0.81      1000
           9       0.81      0.80      0.80      1000

    accuracy                           0.71     10000
   macro avg       0.71      0.71      0.71     10000
weighted avg       0.71      0.71      0.71     10000

